In [1]:
# Import libraries
from bs4 import BeautifulSoup
import FantasyFootball
import numpy as np
# import re
import requests

In [2]:
# Update owners' team abbreviations
owner_ids = {'1': 'Sean Fitzgerald',
          '2': 'Darryl McFarland',
          '3': 'Jason McMillan',
          '5': 'Bradley Putman',
          '7': 'Jason Smith',
          '8': 'Rick Toms',
          '10': 'Jerry LeBlanc',
          '11': 'Matt Zeback',
          '12': 'James Waller',
          '13': 'Zach Hall'}

owner_abbvs = {}
owner_url_preamble = "http://games.espn.com/ffl/clubhouse?leagueId=748024"

for id_num in owner_ids:
    current_url = owner_url_preamble + "&teamId=" + str(id_num)
#     print current_url
    current_abbv = BeautifulSoup(requests.get(current_url).text, 'html.parser').find('h3', {'class': 'team-name'})\
                .find('em').text.strip('(').strip(')')
    owner_abbvs[current_abbv] = owner_ids[id_num]

print "Current Team Name Abbreviations:\n" + "".join(["=" for i in range(32)])
for abbv, name in owner_abbvs.items():
    print '{:20}{:10}'.format(name, abbv)

Current Team Name Abbreviations:
Jason Smith         LB        
Jason McMillan      JM        
James Waller        WALL      
Jerry LeBlanc       LEBL      
Zach Hall           WFF       
Sean Fitzgerald     FITZ      
Rick Toms           QD        
Bradley Putman      BRAD      
Darryl McFarland    TMBs      
Matt Zeback         Z         


In [3]:
action_keywords = {'drafted': '+', 'added': '+', 'dropped': '-', 'traded': '-'}

def updateRosters(url):
    activity_html = BeautifulSoup(requests.get(url).text, 'html.parser').find(id='content').find('table')
    activity_table = list(activity_html)[5:]

    for row in activity_table[:20]:
        date_, type_, detail_, action_ = row
        if "trade" not in type_.text.lower() or "upheld" in type_.text.lower():

            for sub_row in str(detail_).split('<br/>'): 
                if sub_row[:3] != "<td":
                    sub_row = "<td>" + sub_row

                if sub_row[-5:] != "</td>":
                    sub_row += "</td>"

                sub_row = BeautifulSoup(sub_row, 'html.parser')
                owner = owner_abbvs[sub_row.text.split(" ")[0]]
                transaction = sub_row.text.split(" ")[1]
                add_drop = action_keywords[transaction]
                player = sub_row.find('b').text

                print '{:20}{:5}{:20}'.format(owner, add_drop, player)

                if transaction == "traded":
                    recipient = owner_abbvs[sub_row.text.split(" ")[-1]]
                    print '{:20}{:5}{:20}'.format(recipient, "+", player)

            print

In [4]:
activity_url = "http://games.espn.com/ffl/recentactivity?leagueId=748024&seasonId=2017" + "&teamId=-1&tranType=-2" + \
        "&activityType=2" + "&startDate=20170827" + "&endDate=20170830"
    
print activity_url + "\n"
updateRosters(activity_url)

http://games.espn.com/ffl/recentactivity?leagueId=748024&seasonId=2017&teamId=-1&tranType=-2&activityType=2&startDate=20170827&endDate=20170830

Zach Hall           +    Giovani Bernard     

Sean Fitzgerald     +    Matt Prater         

Rick Toms           +    Greg Zuerlein       

Jason Smith         +    Giants D/ST         

Darryl McFarland    +    Julius Thomas       

Bradley Putman      +    Chris Boswell       

Jerry LeBlanc       +    Jason Witten        

Matt Zeback         +    Kevin White         

James Waller        +    Cairo Santos        

James Waller        +    Jaguars D/ST        

Matt Zeback         +    Ravens D/ST         

Jerry LeBlanc       +    Mason Crosby        

Bradley Putman      +    Josh Doctson        

Darryl McFarland    +    Dan Bailey          

Jason Smith         +    Samaje Perine       

Rick Toms           +    Torrey Smith        

Sean Fitzgerald     +    Panthers D/ST       

Zach Hall           +    Adam Vinatieri      

Jason McM

In [5]:
# Perform regression test using all 2016 transactional data
# 2016 End of season rosters: http://games.espn.com/ffl/leaguerosters?leagueId=748024
# Full 2016 activity log: http://games.espn.com/ffl/recentactivity?leagueId=748024&seasonId=2016&activityType=2&startDate=20160807&endDate=20170912&teamId=-1&tranType=-2

# activity_url = "http://games.espn.com/ffl/recentactivity?leagueId=748024&seasonId=2016" + \
#         "&activityType=2&startDate=20160807&endDate=20161009&teamId=-1&tranType=-2"